In [ ]:
# Webscraping Script for ArtPrice
# Combining Everything

In [11]:
# Import Necessary Libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
import time

import pandas as pd
import time
import os
import requests
import re
import string


In [23]:
# 1. Initialize & Login
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Optional for background mode
driver = webdriver.Chrome(options=options)
driver.maximize_window()

# Login once
email = "artauctionproject57@gmail.com"
password = "Artauctionproject2025!"
logged_in = login_to_artprice(driver, email, password)

if not logged_in:
    print("Login failed. Exiting.")
    driver.quit()
    exit()

Accepted cookies.
Logged in successfully


In [24]:
# 2. Scrape all artist URLs (A–Z)
all_artist_links = []
for letter in ["A"]: # string.ascii_uppercase
    print(f"Scraping letter: {letter}")
    try:
        links = scrape_artist_links(driver, letter)
        links = links[20:60] # Limit to 60 artists per letter
        all_artist_links.extend(links)
        print(f"{len(links)} links found for {letter}")
    except Exception as e:
        print(f"Failed for {letter}: {e}")

Scraping letter: A
Starting https://www.artprice.com/artists/all/A
40 links found for A


In [63]:
# 3. Filter for artists with > 10 artworks
qualified_artists = []
for idx, url in enumerate(all_artist_links):
    print(f"Checking artist {idx+1}/{len(all_artist_links)}")
    try:
        artist_info = get_artist_info_if_has_enough_works(driver, url)
        if artist_info:
            qualified_artists.append(artist_info)
            print(f"{artist_info['name']} has {artist_info['count']} works")
    except Exception as e:
        print(f"Skipping due to error: {e}")
    time.sleep(1)

Checking artist 1/40
Checking artist 2/40
Checking artist 3/40
Checking artist 4/40
Checking artist 5/40
Carl Frederik AAGAARD has 440 works
Checking artist 6/40
Checking artist 7/40
Martin AAGAARD has 80 works
Checking artist 8/40
Carl Frederik AAGAARD has 440 works
Checking artist 9/40
Checking artist 10/40
Checking artist 11/40
Martin AAGAARD has 80 works
Checking artist 12/40
Checking artist 13/40
Checking artist 14/40
Gunnar AAGAARD ANDERSEN has 91 works
Checking artist 15/40
Checking artist 16/40
Carl Trier AAGÅRD has 74 works
Checking artist 17/40
Checking artist 18/40
Checking artist 19/40
Checking artist 20/40
Checking artist 21/40
Checking artist 22/40
Checking artist 23/40
Checking artist 24/40
Checking artist 25/40
Checking artist 26/40
Checking artist 27/40
Ahmed Abdel AAL has 12 works
Checking artist 28/40
Checking artist 29/40
Checking artist 30/40
Checking artist 31/40
Checking artist 32/40
Checking artist 33/40
Checking artist 34/40
Checking artist 35/40
Checking artis

In [67]:
# 4. Scrape all artworks per artist
all_artworks = []
for idx, artist in enumerate(qualified_artists):
    print(f"\n Scraping {artist['name']} ({idx+1}/{len(qualified_artists)})")
    artist_url = artist["url"].rstrip("/") + "/lots/pasts?p=1"
    
    try:
        artwork_links = get_all_artwork_links(driver, artist_url)
        print(f"Found {len(artwork_links)} artworks")

        for url in artwork_links:
            data = scrape_artwork_detail(driver, url)
            flat_data = flatten_artwork_data([data])
            all_artworks.extend(flat_data)
    except Exception as e:
        print(f"Error with {artist['name']}: {e}")
    time.sleep(1)


 Scraping Carl Frederik AAGAARD (1/9)
Error with Carl Frederik AAGAARD: HTTPConnectionPool(host='localhost', port=64561): Max retries exceeded with url: /session/ddf6d999cb32b958a75aab8519be8fb4/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a531e350>: Failed to establish a new connection: [Errno 61] Connection refused'))

 Scraping Martin AAGAARD (2/9)
Error with Martin AAGAARD: HTTPConnectionPool(host='localhost', port=64561): Max retries exceeded with url: /session/ddf6d999cb32b958a75aab8519be8fb4/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a561c460>: Failed to establish a new connection: [Errno 61] Connection refused'))

 Scraping Carl Frederik AAGAARD (3/9)
Error with Carl Frederik AAGAARD: HTTPConnectionPool(host='localhost', port=64561): Max retries exceeded with url: /session/ddf6d999cb32b958a75aab8519be8fb4/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a531d660>: Fa

KeyboardInterrupt: 

In [10]:
# ---- Function 1: Login to ArtPrice ----
def login_to_artprice(driver, email, password):
    
    driver.get("https://www.artprice.com")
    time.sleep(5)
    
    # Accept Cookies
    try:
        accept_cookies = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.sln-accept-cookies"))
        )
        accept_cookies.click()
        print("Accepted cookies.")
    except Exception:
        print("No cookie banner or already accepted.")
    time.sleep(7)

    # Click login button
    try:
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.e2e-login-btn"))
        )
        login_button.click()
    except Exception as e:
        print("Couldn't find login button:", e)
        return False
    
    time.sleep(7)
    
    # Fill login form
    try:
        login_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "login"))
        )
        password_input = driver.find_element(By.ID, "pass")
        
        login_input.clear()
        login_input.send_keys(email)
        password_input.clear()
        password_input.send_keys(password)

        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.e2e-login-submit-btn"))).click()
        time.sleep(5)

        print("Logged in successfully")
        return True

    except Exception as e:
        print("Login failed:", e)
        return False

In [6]:
# ---- Function 2: Extracting All Artist Names and URLs ----
BASE_URL = "https://www.artprice.com"

def scrape_artist_links(driver, letter):
    links = []
    url = f"{BASE_URL}/artists/all/{letter}"
    print(f"Starting {url}")
    driver.get(url)

    try:
        while True:
            time.sleep(2)
            artists = driver.find_elements(By.CSS_SELECTOR, "div.col-xs-6.artist a")
            for a in artists:
                href = a.get_attribute("href")
                if href:
                    links.append(href)

            # Try clicking next
            try:
                next_btn = driver.find_element(By.CSS_SELECTOR, "a.next")
                driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
                driver.execute_script("arguments[0].click();", next_btn)
            except NoSuchElementException:
                break
    except Exception as e:
        print(f"Error during scraping {letter}: {e}")

    return links

In [7]:
# ---- Function 3: Filter Artists with > 10 Artworks ----
def get_artist_info_if_has_enough_works(driver, artist_url, min_works=10):
    driver.get(artist_url)
    time.sleep(2)

    try:
        # Find the div that contains "Find prices (X)"
        elem = driver.find_element(By.XPATH, "//div[contains(text(), 'Find prices')]")
        match = re.search(r'Find prices\s*\((\d+)\)', elem.text)
        if match:
            count = int(match.group(1))
            if count > min_works:
                # Get artist name from the page title or h1
                try:
                    artist_name = driver.find_element(By.TAG_NAME, "h1").text
                except NoSuchElementException:
                    artist_name = artist_url.split("/")[-1]
                return {"name": artist_name, "url": artist_url, "count": count}
    except NoSuchElementException:
        pass  # If the div isn't found, skip this artist

    return None

In [8]:
# ---- Function 4: Extracting all Artwork URLs for each Artist ----
def get_all_artwork_links(driver, start_url):
    all_links = []
    driver.get(start_url)

    while True:
        # Wait for artwork grid to load
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lot-container")))

        # Parse page content
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Extract artwork links
        for container in soup.find_all("div", class_="lot-container"):
            a_tag = container.find("a", class_="sln_lot_show")
            if a_tag:
                href = a_tag.get("href")
                full_url = "https://www.artprice.com" + href
                all_links.append(full_url)

        print(f"Scraped {len(all_links)} artworks so far...")

        # Try to click the "Next" button
        try:
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "li.next_page a.sln-next-page"))
            )
            next_button.click()
            time.sleep(2)  # Give time for the next page to load
        except:
            print("No more pages or 'Next' button not found.")
            break

    return all_links


In [42]:
# ---- Function 5: Extracting all Information from a Single Artwork ----
def scrape_artwork_detail(driver, url):

    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # -- Artist --
    artist_tag = soup.select_one("div.artist a.invisiblelink")
    artist = artist_tag.text.strip() if artist_tag else None

    # -- Title --
    title_tag = soup.select_one("div.lot-title h1")
    title = title_tag.text.strip() if title_tag else None

    # -- Lot Number --
    lot_tag = soup.find("div", class_="block marg marg-t-5")
    lot_number = lot_tag.text.strip() if lot_tag else None

    # -- Description (h2/h3/h4 inside .description) --
    description_tag = soup.find("div", class_="description")
    if description_tag:
        desc_parts = [tag.text.strip() for tag in description_tag.find_all(["h2", "h3", "h4"])]
        description = " | ".join(desc_parts)
    else:
        description = None

    # -- Prices --
    final_price = starting_price = estimate = price_incl_premium = None
    prices_tag = soup.find("div", class_="prices")
    if prices_tag:
        # Final hammer price
        hammer = prices_tag.find("div", class_="price")
        if hammer and "Hammer price" in hammer.text:
            span = hammer.find("span", attrs={"ng-show": re.compile("currency.*eur")})
            final_price = span.text.strip() if span else None

        # Price incl. premium
        premium = prices_tag.find("div", class_="price-tax")
        if premium:
            span = premium.find("span", attrs={"ng-show": re.compile("currency.*eur")})
            price_incl_premium = span.text.strip() if span else None

        # Starting price
        start = prices_tag.find("div", class_="price-start")
        if start:
            span = start.find("span", attrs={"ng-show": re.compile("currency.*eur")})
            starting_price = span.text.strip() if span else None

        # Estimate
        estimate_spans = prices_tag.select("div.price-estim span[ng-show*='currency']")
        if estimate_spans:
            estimate = " - ".join([s.text.strip() for s in estimate_spans if "eur" in s.get("ng-show", "")])

    # -- Sale Info --
    sale_title = sale_date = auction_house = location = None
    sale_tag = soup.find("div", class_="sale")
    if sale_tag:
        sale_title_tag = sale_tag.select_one("div.strong.block i")
        sale_title = sale_title_tag.text.strip() if sale_title_tag else None

        sale_date_tag = sale_tag.select_one("div.block.sale-date")
        sale_date = sale_date_tag.text.strip() if sale_date_tag else None

        auction_house_tag = sale_tag.select_one("div.block.strong div")
        auction_house = auction_house_tag.text.strip() if auction_house_tag else None

        # Get last meaningful block for location
        blocks = sale_tag.find_all("div", class_="block")
        for block in reversed(blocks):
            text = block.get_text(strip=True)
            if text and not any(kw in text.lower() for kw in ["sale", "auctioneer", "date"]):
                location = text
                break

    # -- Additional Details --
    additional_details = {}
    datas_tag = soup.select_one("div.sale div.datas")
    if datas_tag:
        for block in datas_tag.find_all("div", class_="block"):
            head_span = block.find("span", class_="head")
            if head_span:
                label = head_span.text.strip().rstrip(":")
                value = block.get_text(separator=" ", strip=True).replace(f"{label}:", "").strip()
                additional_details[label] = value
            else:
                # Try long content (like exhibition blocks)
                content = block.get_text(separator=" ", strip=True)
                if content:
                    key = f"Note {len(additional_details)+1}"
                    additional_details[key] = content

    return {
        "URL": url,
        "Artist": artist,
        "Title": title,
        "Lot Number": lot_number,
        "Description": description,
        "Final Price (EUR)": final_price,
        "Price Incl. Premium (EUR)": price_incl_premium,
        "Starting Price (EUR)": starting_price,
        "Estimate (EUR)": estimate,
        "Sale Title": sale_title,
        "Sale Date": sale_date,
        "Auction House": auction_house,
        "Location": location,
        "Additional Details": additional_details
    }

def flatten_artwork_data(records):
    flat_records = []

    for record in records:
        flat = record.copy()
        details = flat.pop("Additional Details", {})  # Remove nested dict and store keys
        for key, value in details.items():
            flat[f"Detail - {key}"] = value  # Prefix to avoid column name clashes
        flat_records.append(flat)

    return flat_records
